<h1 style='text-align: center'>SQL Queries</h1>

## Getting Data From A SQL Database

### The Structure of a SQL Query

<img src='images/sql_statement.jpg'/>

#### SELECT

- What columns of data do we want? 
- Specify by column names or * (All) 

- Where are we getting our data from? 
- Specify by table name or use join statement to join multiple tables

##### JOIN 

- Use to join multiple tables 
- SELECT * FROM students join favorite_foods on students.id = favorite_foods.student_id
- SELECT * FROM students as s join favorite_foods as f on s.id = f.student_id

#### WHERE

- apply some filter to the data you are querying 
- define criteria for data that should selected from the database 
- SELECT * from students WHERE age > 10 

#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city

#### HAVING

- Use to apply filter AFTER a GROUP BY based on aggregate criteria 
- Where is applied for conditions prior to the Group BY, Having is applied afterwards

For example, if we had a table of student names and the courses they were taking, we could ask a question such as which classes have 3 or more students with the name Matt?

Such a query would look something like this:

```SQL
select class,
       count(student_name) as number_of_matts
       FROM student_courses
       WHERE student_name = "Matt"
       Group By 1
       HAVING count(student_name) >= 3;
```

In [3]:
import sqlite3

conn = sqlite3.connect('tutorial.db')
c = conn.cursor()

In [4]:
columns = [x[0] for x in c.execute('select * from students').description]

In [5]:
columns

['name',
 'birthdate',
 'siblings',
 'birth_place',
 'years_in_nyc',
 'favorite_food']

### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?


1. What are the names of all of the students.

> This is a straightforward review question meant to build some repetition and easy practice.

In [6]:
c.execute('''select name from students''').fetchall()

[('Sean Abu Wilson',),
 ('David Miller',),
 ('Abhijeet Kamble',),
 ('Samantha Jackson',),
 ('Anmol Srivats',),
 ('Ran Tokman',),
 ('Amy Li',),
 ('Florencia Leoni',),
 ('Austin Krause',),
 ('Natalie Overchuk',),
 ('Akshay Sharma',),
 ('Mohamad Eldebek',),
 ('Adam Dick',),
 ('Menachi Korn',),
 ('Miguel Peña',),
 ('Atiar Rahman',),
 ('Fhel Dimaano',),
 ('Maks Pazuniak',),
 ('Alex Mitrani',),
 ('Yish Lim',),
 ('Nicole Roach',),
 ('Omer Hakim',)]

2. Which student has the most siblings?

> This is great place to use a subquery. Encourage students who are initially struggling with a question along the lines of "How could you select the largest number of siblings that anyone has in the group?" From there, you can further push students with a hint if needed: "How can you now make a selection using the result of this, [embedded as a subquery]?"

In [7]:
c.execute("""SELECT name, siblings
FROM students
WHERE siblings = (SELECT MAX(siblings) FROM students);""").fetchall()
[('Florencia Leoni', 4),
 ('Mohamad Eldebek', 4),
 ('Menachi Korn', 4),
 ('Miguel Peña', 4)]

[('Florencia Leoni', 4),
 ('Mohamad Eldebek', 4),
 ('Menachi Korn', 4),
 ('Miguel Peña', 4)]

3. How many students are only children?

> This is considerably easier then question 2. Students can simply place a selection filtered by a where clause. A simple aggregate is also required.

In [8]:
c.execute("""SELECT COUNT(name)
FROM students
WHERE siblings = 0;""").fetchone()

(3,)

4. Which 3 students have lived in NYC the shortest amount of time? [How long has each lived in NYC?]
    > Also considerably easier then the previous two questions. This problem requires a simple order by and limit clauses.
    

In [9]:
c.execute('''SELECT name, years_in_nyc from students order by 2 limit 3
''').fetchall()

[('Anmol Srivats', 0.05), ('Natalie Overchuk', 0.1), ('Austin Krause', 0.17)]

5. How many students are native New Yorkers?

> This place can be tricky depending on how the problem is formulated. For one, are we talking New York City or New York State? If we mean the former, NYC, then the answer below actually would include other non-relevant results. 

It can also be worth noting the difference between like and ilike in this context; ilike being case sensitive while like will ignore all caps. If you wish to further progress this conversation, you can also ask students how they might compare the functionality of like and ilike using python string methods.

In [13]:
c.execute('''select  name, birth_place from Students where birth_place like '%NY' ''').fetchall()


[('David Miller', 'New York, NY'),
 ('Amy Li', 'New York, NY'),
 ('Akshay Sharma', 'New York, NY'),
 ('Adam Dick', 'New York, NY'),
 ('Alex Mitrani', 'New York, NY'),
 ('Nicole Roach', 'Brooklyn, NY')]

6. Do any two students have the same favorite food?

This problem employs the `Having` clause.  Be sure to review the difference between the where and having clause here. (Where filters apply before the group by clause and conditions following the having clause are filters applied after the group by on the resulting aggregate [statistics].) A useful example in doing so, could be to modify the question to something with an additional filtering criterion such as 'do any native new yorkers have the same favorite food?' This would force students to use a where clause prior to the group by to filter the results. Alternatively, see the question below for an alternative but related problem on favorite foods.

In [14]:
c.execute("""SELECT favorite_food, count(favorite_food)
FROM students
GROUP BY favorite_food
HAVING count(favorite_food) > 1
""").fetchall()

[('pizza', 2), ('steak', 2)]

## More Questions

What are the favorite foods of this classroom?

In [12]:
c.execute("""select favorite_food, count(*) from students group by 1 order by count(*) desc;""").fetchall()

[('pizza', 2),
 ('steak', 2),
 ('Avocado', 1),
 ('Biriyani', 1),
 ('Everything Bagels', 1),
 ('Fusilli Sorrentina', 1),
 ('Kare Kare', 1),
 ('Reeses Puffs', 1),
 ('Sushi', 1),
 ('Tabouleh', 1),
 ('burgers', 1),
 ('cheesecake', 1),
 ('chocolate chip cookies', 1),
 ('dumplings', 1),
 ('falafel', 1),
 ('guacamole', 1),
 ('noodles', 1),
 ('pasta', 1),
 ('pho', 1),
 ('rice', 1)]

7. Which student was born closest to the cohort's graduation date?

> This question is in development. It attempts to show some of the difficulties surrounding poorly formatted dates but may be too problematic as an exercise.

In [16]:
c.execute('''
SELECT name,
abs(julianday('2018-' || substr(birthdate,1,2) || '-' || substr(birthdate,4,2)) - julianday('2018-06-19'))
FROM students
ORDER BY 2
LIMIT 1
''').fetchall() 

[('Nicole Roach', 3.0)]